In [27]:
# Imports

import tkinter as tk
import subprocess
import os
from tkinter import ttk
import networkx as nx
import numpy as np
import itertools

from AC.AC_merger import merge_AC_types

from Graph_generators.graph_generator import generate_graphs_edges
from Graph_visualizers.graph_plot import plot_graph
from Graph_visualizers.networkx_graph_constructor import construct_graph

from utils.POINTS_to_dic import retrieve_nodes
from utils.EDGES_to_dic import retrieve_edges
from utils.HTML_to_arr import retrieve_edges_from_html
from utils.nodes_parity import establish_parity_degree
from utils.filter import eliminate_duplicate_edges

from Kruskal.kruskal_stepwise import kruskal_stepwise
from Blossom.min_weight_matching import min_weight_matching
from Hierholzer.eulerian_circuit import eulerian_circuit
from Shortcutting.shortcutting import shortcutting
from two_opt.optimize_tour import optimize_tour

In [28]:
# Some beautiful colors for the graphs

def color_generator():
    colors = ['#b694fa', # purple
              '#87cefa', # light blue
              '#94faca', # turquoise
              '#f2d9a6', # beige
              '#8cf471', # light green
              '#af002a', # crimson
              '#6474ed', # blue
              '#2b4fe2', # dark blue
              '#e22b8a', # dark pink
              '#f0e58c', # gold
              '#008080', # teal
              '#ffb6e4', # light pink
              '#ff9d5e', # orange
              '#dcdcdc', # gray
              '#693ac1', # dark purple
              ]
    return itertools.cycle(colors)

# Example usage:
# gen = color_generator()
# color = next(gen)

In [29]:
python_executable = "C:/Users/Alexis/AppData/Local/anaconda3/python.exe"

In [30]:
# Functions used for centering window and creating custom buttons

def center_window(window):
    
    '''This function centers the given Tkinter window on the screen.
    It calculates the width and height of the window, as well as the screen dimensions,
    and sets the window's geometry accordingly. The window will be positioned in the center of the screen.
    '''
    
    window.update_idletasks()
    
    # width = window.winfo_reqwidth()
    # height = window.winfo_reqheight()
    
    # Retrieve the width and height of the window after it has been shown
    width = window.winfo_width()
    height = window.winfo_height()
    
    # Calculate the x and y coordinates to center the window on the screen
    x = (window.winfo_screenwidth() // 2) - (width // 2)
    y = (window.winfo_screenheight() // 2) - (height // 2)
    
    # Set the geometry of the window to the calculated size and position
    window.geometry(f'{width}x{height}+{x}+{y}')
    
#===========================================================================================================================
    
def create_custom_button(text, command, card_frame):
    
    '''This function creates a custom button with a specific text and command.
    The button has a solid border, a specific background color, and a hover effect.
    The button is placed inside a given card frame.
    '''
    
    # Create a button with the specified text and command
    button = tk.Button(
        card_frame, text=text, command=command, 
        bg="#212121", fg="#fff", borderwidth=2, relief="solid", 
        font=("Arial", 12), padx=20, pady=10
    )
    
    # The two clickable buttons
    button.bind("<Enter>", lambda e: button.config(bg="#303030"))
    button.bind("<Leave>", lambda e: button.config(bg="#212121"))
    
    return button

In [31]:
# First, we ask wether we want to generate new points manually or not

def generating_points_question():
    
    '''This function creates a Tkinter window that asks the user whether they want to generate points manually or not.
    If the user clicks "Yes", the function returns True, and if they click "No", it returns False.
    The window is centered on the screen and contains a question label and two buttons for the user's response.
    '''
    
    # Initialize a variable to hold the result
    gen = None

    # Window to ask whether we want to generate new points manually or not
    generate_points_wd = tk.Tk()
    generate_points_wd.title("Generate points")
    generate_points_wd.configure(bg="#f4f4f3")

    # Define temporary setting for the window
    generate_points_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(generate_points_wd, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        circle = tk.Label(tools_frame, bg=color, width=2, height=1, bd=0)
        circle.pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Do you want to generate points manually ?", bg="#f4f4f3", font=("Arial", 14))
    question.pack(anchor="center", pady=20)

    #---------------------------------------------------------------------------------------------------------------------------

    # Define the button actions that set the result variable
    def on_generate_button():
        nonlocal gen
        gen = True                    # Yes, we want to generate points manually, set gen to True
        generate_points_wd.destroy()  # Close the window after clicking

    def on_do_not_generate_button():
        nonlocal gen
        gen = False                   # No, we do not want to generate points manually, set gen to False
        generate_points_wd.destroy()  # Close the window after clicking

    # Yes button to generate points manually
    generate_button = create_custom_button("Yes", on_generate_button, card_frame)
    generate_button.pack(side=tk.LEFT, padx=40)

    # No button to not generate points manually
    do_not_generate_button = create_custom_button("No", on_do_not_generate_button, card_frame)
    do_not_generate_button.pack(side=tk.RIGHT, padx=40)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Resizes the window to fit the content
    generate_points_wd.update_idletasks()
    generate_points_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")
    
    # Center the window
    center_window(generate_points_wd)

    # Main loop
    generate_points_wd.mainloop()

    # Return the result after the window is closed
    return gen

# Generating points manually (POINTS folder)

In [32]:
# If we want to generate points manually, we ask which map to use

# Function to generate points and store selections
def select_map():
    
    '''This function creates a Tkinter window that allows the user to select a map from a dropdown menu.
    The user can select a map and click the "Submit" button to confirm their selection.
    The selected map is stored and returned in an array.
    The window is centered on the screen and contains a question label, a dropdown menu, and a submit button.
    '''
    
    # Array to store selected maps
    selected_maps = []

    # Window to ask which map to use to generate points
    choose_map_wd = tk.Tk()
    choose_map_wd.title("Select a Map")
    choose_map_wd.configure(bg="#f4f4f3")
    
    # Define temporary setting for the window
    choose_map_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(choose_map_wd, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        circle = tk.Label(tools_frame, bg=color, width=2, height=1, bd=0)
        circle.pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Select a map to generate points:", bg="#f4f4f3", font=("Arial", 14))
    question.pack(pady=30)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Get list of JPG files in MAPS folder
    maps_folder = "Point_maker/MAPS"
    map_files = [f for f in os.listdir(maps_folder) if f.endswith('.jpg') or f.endswith('.JPG')]
    
    # Create a dropdown menu with the list of maps
    selected_map = tk.StringVar()
    dropdown = ttk.Combobox(card_frame, textvariable=selected_map, values=map_files)
    dropdown.place(relx=0.5, rely=0.5, anchor="center")
    
    # Ensure the value is set correctly by binding the dropdown to a function
    def update_selection(event):
        selected = dropdown.get()
        if selected not in selected_maps:
            selected_maps.append(selected)
    
    dropdown.bind("<<ComboboxSelected>>", update_selection)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Modify submit button to close window and return the selected maps
    submit_button = tk.Button(
        card_frame, text="Submit", 
        command=lambda: choose_map_wd.destroy(),  # Close the window
        bg="#212121", fg="#fff", borderwidth=2, relief="solid", 
        font=("Arial", 12), padx=20, pady=10
    )
    submit_button.bind("<Enter>", lambda e: submit_button.config(bg="#303030"))
    submit_button.bind("<Leave>", lambda e: submit_button.config(bg="#212121"))
    submit_button.pack(pady=10)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Resizes the window to fit the content
    choose_map_wd.update_idletasks()
    choose_map_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")
    
    # Center the window
    center_window(choose_map_wd)

    # Main loop
    choose_map_wd.mainloop()

    # Return the array of selected maps
    return selected_maps

In [33]:
# Once the map is chosen, we use the code in the Point_maker folder to generate points
# right-click on the map to place a red point and enter the name of it
# use z,q,w,s to move the map and +,- to zoom in and out
# Press 'p' to save the points you already placed. Those points now appear in blue
# Press 'y' to stop the process and save the points in the POINTS folder

def place_point_fct(selected_value, python_executable, output_folder):
    
    '''This function generates points on a map using the Point_maker script.
    It takes the selected map, the Python executable path, and the output folder as arguments.
    
    Right-click on the map to place a red point and enter the name of it.
    Use z, q, w, s to move the map and +, - to zoom in and out.
    Press 'p' to save the points you already placed. Those points now appear in blue.
    Press 'y' to stop the process and save the points in the POINTS folder.
    
    The points are stored in a .txt file in the specified output folder.
    '''
    
    # Run the code in the Point_maker folder
    script_path = "Point_maker/Point_Maker.py"
    map_name = f"Point_maker/MAPS/{selected_value}"
    command = [python_executable, script_path, map_name, output_folder]
    subprocess.run(command, check=True)
    
    # Remove map files created during the process
    os.remove('Map_with_blue_points.JPG')
    os.remove('Map_with_red_points.JPG')

# Generating points using AC database (POINTS folder)

In [34]:
# If we don't want to generate points manually, we ask if we want to use the AC database to generate points

def use_AC_question():
    
    '''This function creates a Tkinter window that asks the user whether they want to use the AC database to generate points.
    If the user clicks "Yes", the function returns True, and if they click "No", it returns False.
    The window is centered on the screen and contains a question label and two buttons for the user's response.
    
    If answered no, the user is expected to already have a points.txt file to work with.
    '''
    
    # Initialize a variable to hold the result
    use_AC = None
    
    # Window to ask wether we want to use the AC database to generate points
    use_AC_wd = tk.Tk()
    use_AC_wd.title("Use AC")
    use_AC_wd.configure(bg="#f4f4f3")

    # Define temporary setting for the window
    use_AC_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(use_AC_wd, width=380, height=250, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        circle = tk.Label(tools_frame, bg=color, width=2, height=1, bd=0)
        circle.pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Do you want to use AC database ?", bg="#f4f4f3", font=("Arial", 14))
    question.pack(pady=30)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Define the button actions that set the result variable
    def on_use_button():
        nonlocal use_AC
        use_AC = True        # Yes, we want to use the AC database, set use_AC to True
        use_AC_wd.destroy()  # Close the window after clicking

    def on_do_not_use_button():
        nonlocal use_AC
        use_AC = False       # No, we do not want to use the AC database, set use_AC to False
        use_AC_wd.destroy()  # Close the window after clicking

    # Yes button to use the AC database
    use_button = create_custom_button("Yes", on_use_button, card_frame)
    use_button.pack(side=tk.LEFT, padx=40)

    # No button to not use the AC database
    do_not_use_button = create_custom_button("No", on_do_not_use_button, card_frame)
    do_not_use_button.pack(side=tk.RIGHT, padx=40)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Resizes the window to fit the content
    use_AC_wd.update_idletasks()
    use_AC_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")
    
    # Center the window
    center_window(use_AC_wd)

    # Main loop
    use_AC_wd.mainloop()

    # Return the result after the window is closed
    return use_AC

In [35]:
# If we want to use the AC database, we ask which features of the game we want to use to generate points

# Initialize a global counter and listbox
global_counter = 0  
listbox = None  

# Function to update the global counter when selecting options in the listbox
def on_select(event):
    
    '''This function updates the global counter based on the selected options in the listbox.
    It retrieves the selected options, extracts the numbers from them, and sums them up.
    The updated counter value is displayed in the counter_label.
    '''
    
    global global_counter
    selected = listbox.curselection()
    global_counter = sum(int(option.split('[')[1].strip(']')) for option in [listbox.get(i) for i in selected])
    counter_label.config(text=f"Global counter: {global_counter}")

#===========================================================================================================================

def use_AC():
    
    '''This function creates a Tkinter window that allows the user to select options from a listbox.
    The user can select multiple set of items and combine them to generate points.
    The selected options are stored in an array and returned when the user clicks the "Submit" button.
    The window is centered on the screen and contains a question label, a listbox, and a submit button.
    '''
    
    # Array to store selected options
    selected_options = []

    # Window to ask which features of the game we want to use to generate points
    select_AC_wd = tk.Tk()
    select_AC_wd.title("Select options")
    select_AC_wd.configure(bg="#f4f4f3")
    
    # Define temporary setting for the window
    select_AC_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(select_AC_wd, width=400, height=800, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        circle = tk.Label(tools_frame, bg=color, width=2, height=1, bd=0)
        circle.pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Select your options:", bg="#f4f4f3", font=("Arial", 14))
    question.pack(pady=30)

    #---------------------------------------------------------------------------------------------------------------------------

    # Define options of the dropdown
    options = []
    for file in os.listdir("AC/AC_database"):
        type = file.split("_")[1].split(".")[0]
        nb = file.split("_")[0]
        options.append(f"{type} [{nb}]")
    options = sorted(options, key=lambda x: int(x.split('[')[1].strip(']')))
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Create a scrollbar
    scrollbar = tk.Scrollbar(card_frame, orient="vertical")
    scrollbar.pack(side="right", fill="y")

    # Create a listbox
    global listbox
    listbox = tk.Listbox(card_frame, selectmode="multiple", yscrollcommand=scrollbar.set, font=("Arial", 12), bg="#f4f4f3")
    listbox.pack(side="left", fill="both", expand=True)
    
    # Attach scrollbar to the listbox
    scrollbar.config(command=listbox.yview)

    # Add options to the listbox
    for option in options:
        listbox.insert("end", option)
    
    # Bind the select event to the on_select function
    listbox.bind('<<ListboxSelect>>', on_select)
    
    # Label for the global counter
    global counter_label
    counter_label = tk.Label(card_frame, text="Global counter: 0", bg="#f4f4f3", font=("Arial", 12))
    counter_label.pack(pady=10)
    
    # Submit button that closes the window and stores the selected options
    def store_selection():
        selected_indices = listbox.curselection()
        for i in selected_indices:
            selected_options.append(listbox.get(i))
        select_AC_wd.destroy()  # Close the window

    submit_button = tk.Button(
        card_frame, text="Submit", command=store_selection,
        bg="#212121", fg="#fff", borderwidth=2, relief="solid", 
        font=("Arial", 12), padx=20, pady=10
    )
    submit_button.bind("<Enter>", lambda e: submit_button.config(bg="#303030"))
    submit_button.bind("<Leave>", lambda e: submit_button.config(bg="#212121"))
    submit_button.pack(pady=10)
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Resizes the window to fit the content
    select_AC_wd.update_idletasks()
    select_AC_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")

    # Center the window
    center_window(select_AC_wd)

    # Main loop
    select_AC_wd.mainloop()

    # Return the array of selected options
    return selected_options


# Generate edges corresponding to types of graphs (EDGES folder)

In [36]:
# Choose the set of points to use based on the method used to generate them before

def select_points(points_folder):
    
    '''This function creates a Tkinter window that allows the user to select a set of points from a dropdown menu.
    The user can select a points set in the points_folder and click the "Submit" button to confirm their selection.
    The selected points set is stored and returned as a string.
    The window is centered on the screen and contains a question label, a dropdown menu, and a submit button.
    '''
    
    # Window setup
    select_points_wd = tk.Tk()
    select_points_wd.title("Select points")
    select_points_wd.configure(bg="#f4f4f3")
    
    # Define temporary setting for the window
    select_points_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(select_points_wd, width=380, height=250, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        tk.Label(tools_frame, bg=color, width=2, height=1, bd=0).pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Select a points set:", bg="#f4f4f3", font=("Arial", 14))
    question.pack(pady=30)
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Get list of TXT files in POINTS folder
    points_files = [f for f in os.listdir(points_folder) if f.endswith('.txt')]

    # Create a dropdown menu
    selected_points = tk.StringVar()
    dropdown = ttk.Combobox(card_frame, textvariable=selected_points, values=points_files)
    dropdown.pack(pady=10)

    # Ensure the value is set correctly
    dropdown.bind("<<ComboboxSelected>>", lambda event: selected_points.set(dropdown.get()))

    # Function to close window and store selected value
    def store_selection():
        select_points_wd.destroy()

    submit_button = tk.Button(
        card_frame, text="Submit", command=store_selection,
        bg="#212121", fg="#fff", borderwidth=2, relief="solid", 
        font=("Arial", 12), padx=20, pady=10
    )
    submit_button.bind("<Enter>", lambda e: submit_button.config(bg="#303030"))
    submit_button.bind("<Leave>", lambda e: submit_button.config(bg="#212121"))
    submit_button.pack(pady=10)
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Resizes the window to fit the content
    select_points_wd.update_idletasks()
    select_points_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")

    # Center the window
    center_window(select_points_wd)

    # Main loop
    select_points_wd.mainloop()

    # Return the selected points set
    return selected_points.get()


In [37]:
# Choose the graphs to generate with the chosen set of points

def select_graphs():
    
    '''This function creates a Tkinter window that allows the user to select graph types from a list of checkboxes.
    The user can select multiple graph types and click the "Submit" button to confirm their selection.
    The selected graph types are stored in an array and returned when the user clicks the "Submit" button.
    The window is centered on the screen and contains a question label, checkboxes for each graph type, and a submit button.
    '''
    
    # Window setup
    select_graphs_wd = tk.Tk()
    select_graphs_wd.title("Select Graphs")
    select_graphs_wd.configure(bg="#f4f4f3")
    
    # Define temporary setting for the window
    select_graphs_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(select_graphs_wd, width=380, height=450, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        tk.Label(tools_frame, bg=color, width=2, height=1, bd=0).pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Select graph types:", bg="#f4f4f3", font=("Arial", 14))
    question.pack(pady=20)
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Graph options
    options = ["RNG", "Urquhart", "KNN", "Delaunay", "d_radius", "Gabriel", "Complete"]

    # Dictionary to store checkbox variables
    selected_options = {option: tk.BooleanVar() for option in options}
    selected_graphs = []

    def update_selection():
        """Updates selection based on checkboxes."""
        nonlocal selected_graphs
        selected_graphs = [option for option, var in selected_options.items() if var.get()]

    def toggle_all():
        """Toggles all checkboxes based on the 'Select All' option."""
        state = select_all_var.get()
        for var in selected_options.values():
            var.set(state)
        update_selection()

    # "Select All" checkbox
    select_all_var = tk.BooleanVar()
    select_all_checkbox = tk.Checkbutton(card_frame, text="Select All", variable=select_all_var, bg="#f4f4f3",
                                         font=("Arial", 12), command=toggle_all)
    select_all_checkbox.pack(pady=5)

    # Create checkboxes for each option
    for option in options:
        checkbox = tk.Checkbutton(card_frame, text=option, variable=selected_options[option], bg="#f4f4f3",
                                  font=("Arial", 12), command=update_selection)
        checkbox.pack(pady=2)

    def store_selection():
        """Closes window and stores the selected options."""
        update_selection()
        select_graphs_wd.destroy()

    submit_button = tk.Button(
        card_frame, text="Submit", command=store_selection,
        bg="#212121", fg="#fff", borderwidth=2, relief="solid",
        font=("Arial", 12), padx=20, pady=10
    )
    submit_button.bind("<Enter>", lambda e: submit_button.config(bg="#303030"))
    submit_button.bind("<Leave>", lambda e: submit_button.config(bg="#212121"))
    submit_button.pack(pady=20)
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Resizes the window to fit the content
    select_graphs_wd.update_idletasks()
    select_graphs_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")
    
    # Center the window
    center_window(select_graphs_wd)
    
    # Main loop
    select_graphs_wd.mainloop()

    # Return the selected graph types
    return selected_graphs

# Select things to save

In [38]:
# Choose the things to save during the algorithms
def select_things_to_save():
    
    # Window setup
    select_things_to_save_wd = tk.Tk()
    select_things_to_save_wd.title("Select things to save")
    select_things_to_save_wd.configure(bg="#f4f4f3")
    
    # Define temporary setting for the window
    select_things_to_save_wd.geometry("1000x1000")

    # Create card frame
    card_frame = tk.Frame(select_things_to_save_wd, width=380, height=450, bg="#f4f4f3", bd=2, relief="ridge")
    card_frame.place(relx=0.5, rely=0.5, anchor="center")

    # Create tools bar
    tools_frame = tk.Frame(card_frame, bg="#454a50")
    tools_frame.place(relx=0.5, y=-12, anchor="n")

    for color in ["#ff605c", "#ffbd44", "#00ca4e"]:
        tk.Label(tools_frame, bg=color, width=2, height=1, bd=0).pack(side=tk.LEFT, padx=2, pady=4)

    # Add a question label to the card frame
    question = tk.Label(card_frame, text="Select things to save:", bg="#f4f4f3", font=("Arial", 14))
    question.pack(pady=20)
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Graph options
    options = ["Initial graphs (GRAPH)",                         # To store the initial graphs
               
               "Kruskal process (GIF)",                          # To store the GIF of the kruskal algo
               "Kruskal MST (GRAPH)",                            # To store the MST outputed by kruskal
               
               "Odd degree nodes graphs (GRAPH)",                # To store the odd degree nodes graphs
               "Blossom MWPM (GRAPH)",                           # To store the matching on odd degree nodes
               "Superposition MST and MWPM (GRAPH)",             # To store the superposition of the MST and the matching
               # "GIF_blossom_matching_process",                 # To store the GIF of the blossom algo
               
               "Hierholzer process (GIF)",                       # To store the GIF of the Hierholzer algo
               "Hierholzer tour (GRAPH)",                        # To store the eulerian circuit outputed by Hierholzer
               
               "Shortcutting process (GIF)",                     # To store the GIF of the shortcutting process
               "Shortcutting tour (GRAPH)",                      # To store the circuit outputed by shortcutting
               
               "2-opt process (GIF)",                            # To store the GIF of the 2-opt process
               "Final tour (GRAPH)"]                             # To store the final circuit outputed by 2-opt
    
    #---------------------------------------------------------------------------------------------------------------------------

    # Dictionary to store checkbox variables
    selected_options = {option: tk.BooleanVar() for option in options}
    selected_things_to_save = []

    def update_selection():
        """Updates selection based on checkboxes."""
        nonlocal selected_things_to_save
        selected_things_to_save = [option for option, var in selected_options.items() if var.get()]

    def toggle_all():
        """Toggles all checkboxes based on the 'Select All' option."""
        state = select_all_var.get()
        for var in selected_options.values():
            var.set(state)
        update_selection()

    # "Select All" checkbox
    select_all_var = tk.BooleanVar()
    select_all_checkbox = tk.Checkbutton(card_frame, text="Select All", variable=select_all_var, bg="#f4f4f3",
                                         font=("Arial", 12), command=toggle_all)
    select_all_checkbox.pack(pady=5)

    # Create checkboxes for each option
    for option in options:
        checkbox = tk.Checkbutton(card_frame, text=option, variable=selected_options[option], bg="#f4f4f3",
                                  font=("Arial", 12), command=update_selection)
        checkbox.pack(pady=2)

    def store_selection():
        """Closes window and stores the selected options."""
        update_selection()
        select_things_to_save_wd.destroy()

    submit_button = tk.Button(
        card_frame, text="Submit", command=store_selection,
        bg="#212121", fg="#fff", borderwidth=2, relief="solid",
        font=("Arial", 12), padx=20, pady=10
    )
    submit_button.bind("<Enter>", lambda e: submit_button.config(bg="#303030"))
    submit_button.bind("<Leave>", lambda e: submit_button.config(bg="#212121"))
    submit_button.pack(pady=20)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Resizes the window to fit the content
    select_things_to_save_wd.update_idletasks()
    select_things_to_save_wd.geometry(f"{card_frame.winfo_width() + 50}x{card_frame.winfo_height() + 50}")

    # Center the window
    center_window(select_things_to_save_wd)
    
    # Main loop
    select_things_to_save_wd.mainloop()

    # Return the selected things to save
    return selected_things_to_save

# Output folders and colors

In [39]:
# Variables for the things to save
# Modify only the folders and colors if you want, don't touch the bools !!!

# Create a txt file for the cleaner with the names of the folders to delete
with open("folders_to_delete.txt", "w") as f:
    f.write("")

#---------------------------------------------------------------------------------------------------------------------------

# To save the points txt files
points_folder = 'POINTS'

# To save the edges txt files
edges_folder = 'EDGES'

# To save the initial graphs
save_initial_graphs_bool   = None
save_initial_graphs_folder = 'INITIAL_GRAPHS'

#---------------------------------------------------------------------------------------------------------------------------

# For the cleaner
with open("folders_to_delete.txt", "a") as f:
    f.write(points_folder + "\n")
    f.write(edges_folder + "\n")
    f.write(save_initial_graphs_folder + "\n")
    
#---------------------------------------------------------------------------------------------------------------------------

# For display purposes, default values for the graphs
display = {}
display['node_size'] = 100
display['node_color'] = 'skyblue'
display['edge_color'] = 'lightgray'
display['font_size'] = 5

In [40]:
# Kruskal algo details

# Upper folder to store all results about kruskal
# don't use 'kruskal' or 'KRUSKAL' alone as it's the folder of the codes
upper_folder_kruskal = 'KRUSKAL_RESULTS'

# Color to display the accepted edges during the kruskal algo
# Color to display the rejected edges during the kruskal algo
accepted_edges_color_code_kruskal = '#af002a' # crimson
rejected_edges_color_code_kruskal = '#2b4fe2' # dark blue

#---------------------------------------------------------------------------------------------------------------------------

# To save the progress of the kruskal algo as GIF
save_kruskal_gif_bool    = None
save_kruskal_gif_folder  = upper_folder_kruskal + '/GIF_KRUSKAL_PROCESS'

# To save the progress of the kruskal algo as HTML page
save_kruskal_html_bool   = None
save_kruskal_html_folder = upper_folder_kruskal + '/HTML_KRUSKAL_PROCESS'

# To save the MST outputed by kruskal algo as a graph
save_kruskal_MST_bool    = None
save_kruskal_MST_folder  = upper_folder_kruskal + '/GRAPH_KRUSKAL_MST'

# To save the MST outputed by kruskal algo as HTML page
save_kruskal_MST_html_bool   = None
save_kruskal_MST_html_folder = upper_folder_kruskal + '/HTML_KRUSKAL_MST'

#---------------------------------------------------------------------------------------------------------------------------

# For the cleaner
with open("folders_to_delete.txt", "a") as f:
    f.write(save_kruskal_gif_folder + "\n")
    f.write(save_kruskal_html_folder + "\n")
    f.write(save_kruskal_MST_folder + "\n")
    f.write(save_kruskal_MST_html_folder + "\n")
    f.write(upper_folder_kruskal + "\n")

In [41]:
# Blossom algo details

# Upper folder to store all results about the blossom algo
# don't use 'blossom' or 'BLOSSOM' alone as it's the folder of the codes
upper_folder_blossom = 'BLOSSOM_RESULTS'

# Color to display the S vertices/blossom during the blossom algo
S_color_code_blossom = '#b694fa' # purple
# Color to display the T vertices/blossom during the blossom algo
T_color_code_blossom = '#2b4fe2' # dark blue
# Color to display the edges of the matching during the blossom algo
matching_color_code_blossom = '#ff605c' # red
# Color to display the blossoms during the blossom algo
blossom_color_code_blossom = '#94faca' # turquoise
# Color to display the visited vertices during the blossom algo
visited_color_code_blossom = '#ff9d5e' # orange

#---------------------------------------------------------------------------------------------------------------------------

# To save the odd degree nodes graphs
save_blossom_odd_degree_graphs_bool   = None
save_blossom_odd_degree_graphs_folder = upper_folder_blossom + '/ODD_DEGREE_GRAPHS'

# To save the evolution of the matching of odd degree nodes as GIF
save_blossom_gif_bool   = None
save_blossom_gif_folder = upper_folder_blossom + '/GIF_BLOSSOM'

# To save the evolution of the matching of odd degree nodes as HTML page
save_blossom_html_bool   = None
save_blossom_html_folder = upper_folder_blossom + '/HTML_BLOSSOM'

# To save the resulting matching of odd degree nodes as a graph
save_blossom_matching_bool   = None
save_blossom_matching_folder = upper_folder_blossom + '/MATCHING_BLOSSOM'

# To save the resulting matching of odd degree nodes as HTML page
save_blossom_matching_html_bool   = None
save_blossom_matching_html_folder = upper_folder_blossom + '/HTML_MATCHING_BLOSSOM'

# To save the superposition of the odd degree nodes graphs and the matching
save_superposition_bool   = None
save_superposition_folder = upper_folder_blossom + '/SUPERPOSITION_BLOSSOM'

#---------------------------------------------------------------------------------------------------------------------------

# For the cleaner
with open("folders_to_delete.txt", "a") as f:
    f.write(save_blossom_odd_degree_graphs_folder + "\n")
    f.write(save_blossom_gif_folder + "\n")
    f.write(save_blossom_html_folder + "\n")
    f.write(save_blossom_matching_folder + "\n")
    f.write(save_blossom_matching_html_folder + "\n")
    f.write(save_superposition_folder + "\n")
    f.write(upper_folder_blossom + "\n")

In [42]:
# Hierholzer algo details

# Upper folder to store all results about the hierholzer algo
# don't use 'hierholzer' or 'HIERHOLZER' alone as it's the folder of the codes
upper_folder_hierholzer = 'HIERHOLZER_RESULTS'

# Color to display the accepted edges during the hierholzer algo
# Color to display the rejected edges during the hierholzer algo
accepted_edges_color_code_hierholzer = '#87cefa' # light blue
rejected_edges_color_code_hierholzer = '#b694fa' # purple

#---------------------------------------------------------------------------------------------------------------------------

# To save the evolution of the eulerian circuit as GIF
save_hierholzer_gif_bool   = None
save_hierholzer_gif_folder = upper_folder_hierholzer + '/GIF_HIERHOLZER_PROCESS'

# To save the evolution of the eulerian circuit as HTML page
save_hierholzer_html_bool   = None
save_hierholzer_html_folder = upper_folder_hierholzer + '/HTML_HIERHOLZER_PROCESS'

# To the eulerian circuit outputed by hierholzer algo as a graph
save_hierholzer_eulerian_circuit_bool   = None
save_hierholzer_eulerian_circuit_folder = upper_folder_hierholzer + '/GRAPH_HIERHOLZER_CIRCUIT'

# To the eulerian circuit outputed by hierholzer algo as HTML page
save_hierholzer_eulerian_circuit_html_bool   = None
save_hierholzer_eulerian_circuit_html_folder = upper_folder_hierholzer + '/HTML_HIERHOLZER_CIRCUIT'

#---------------------------------------------------------------------------------------------------------------------------

# For the cleaner
with open("folders_to_delete.txt", "a") as f:
    f.write(save_hierholzer_gif_folder + "\n")
    f.write(save_hierholzer_html_folder + "\n")
    f.write(save_hierholzer_eulerian_circuit_folder + "\n")
    f.write(save_hierholzer_eulerian_circuit_html_folder + "\n")
    f.write(upper_folder_hierholzer + "\n")

In [43]:
# Shortcutting algo details

# Upper folder to store all results about the shortcutting algo
# don't use 'shortcutting' or 'SHORTCUTTING' alone as it's the folder of the codes
upper_folder_shortcutting = 'SHORTCUTTING_RESULTS'

# Color to display the accepted edges during the shortcutting algo
# Color to display the rejected edges during the shortcutting algo
accepted_edges_color_code_shortcutting = '#008080' # teal
rejected_edges_color_code_shortcutting = '#ffbd44' # yellow

#---------------------------------------------------------------------------------------------------------------------------

# To save the evolution of the shortcutting algo as GIF
save_shortcutting_gif_bool    = None
save_shortcutting_gif_folder  = upper_folder_shortcutting + '/GIF_SHORTCUTTING_PROCESS'

# To save the evolution of the shortcutting algo as HTML page
save_shortcutting_html_bool   = None
save_shortcutting_html_folder = upper_folder_shortcutting + '/HTML_SHORTCUTTING_PROCESS'

# To save the circuit outputed by shortcutting algo as a graph
save_shortcutting_tour_bool   = None
save_shortcutting_tour_folder = upper_folder_shortcutting + '/GRAPH_SHORTCUTTING_TOUR'

# To save the circuit outputed by shortcutting algo as HTML page
save_shortcutting_tour_html_bool   = None
save_shortcutting_tour_html_folder = upper_folder_shortcutting + '/HTML_SHORTCUTTING_TOUR'

#---------------------------------------------------------------------------------------------------------------------------

# For the cleaner
with open("folders_to_delete.txt", "a") as f:
    f.write(save_shortcutting_gif_folder + "\n")
    f.write(save_shortcutting_html_folder + "\n")
    f.write(save_shortcutting_tour_folder + "\n")
    f.write(save_shortcutting_tour_html_folder + "\n")
    f.write(upper_folder_shortcutting + "\n")

In [44]:
# 2-opt algo details

# Upper folder to store all results about the 2-opt algo
# don't use 'two_opt' or 'TWO_OPT' alone as it's the folder of the codes
upper_folder_2opt = 'TWO_OPT_RESULTS'

# Color to display the tour obtained after the 2-opt algo
# Color to display the crossing edges during the 2-opt algo
# Color to display the corrected edges during the 2-opt algo
tour_edges_color_code_2opt      = '#af002a' # crimson
crossing_edges_color_code_2opt  = '#b694fa' # purple
corrected_edges_color_code_2opt = '#008080' # teal

#---------------------------------------------------------------------------------------------------------------------------

# To save the evolution of the 2-opt algo as GIF
save_2opt_gif_bool           = None
save_2opt_gif_folder         = upper_folder_2opt + '/GIF_2OPT_PROCESS'

# To save the evolution of the 2-opt algo as HTML page
save_2opt_html_bool          = None
save_2opt_html_folder        = upper_folder_2opt + '/HTML_2OPT_PROCESS'

#---------------------------------------------------------------------------------------------------------------------------

# To save the final tour outputed by 2-opt algo as a graph
save_2opt_final_graph_bool   = None
save_2opt_final_graph_folder = 'FINAL_RESULTS' + '/FINAL_GRAPHS'

# To save the final tour outputed by 2-opt algo as HTML page
save_2opt_final_graph_html_bool   = None
save_2opt_final_graph_html_folder = 'FINAL_RESULTS' + '/HTML_FINAL_GRAPHS'

#---------------------------------------------------------------------------------------------------------------------------

# For the cleaner
with open("folders_to_delete.txt", "a") as f:
    f.write(save_2opt_gif_folder + "\n")
    f.write(save_2opt_html_folder + "\n")
    f.write(save_2opt_final_graph_folder + "\n")
    f.write(save_2opt_final_graph_html_folder + "\n")
    f.write(upper_folder_2opt + "\n")
    f.write("FINAL_RESULTS" + "\n")

# Main code

In [45]:
# Generate points in the points_folder

# Ask if we want to generate new points manually or not
gen = generating_points_question()

# If so...
if gen == True:
    
    # Choose a map on which to add points manually
    selected_maps = select_map()
    # Execute the code in the Point_maker folder to generate points file in the points_folder
    place_point_fct(selected_maps[0], python_executable, points_folder)
    
    # Save the memory by deleting the variables
    del selected_maps, gen
    
#---------------------------------------------------------------------------------------------------------------------------

# Otherwise...    
elif gen == False:
    
    # Ask if we want to use the AC database to generate points
    use = use_AC_question()
    
    # If so...
    if use == True:
        
        # Choose the features of the game to use to generate points
        selected_options = use_AC()
        # Merge the selected features to generate the points file in the points_folder
        merge_AC_types(selected_options, points_folder)
        
        # Save the memory by deleting the variables
        del selected_options
        
    # Save the memory by deleting the variables
    del use, gen
        
# Otherwise, we assume that there already exists a points file in the points_folder

In [46]:
# Generate graphs in the edges_folder

# Ask the user to select the set of points to use in the points_folder
selected_points = select_points(points_folder)
# Ask the user to select the graphs to generate with the selected set of points
selected_graphs = select_graphs()

# A bool to state if 'Complete' is in the selected graphs or not
complete_in_selected_graphs_bool = True if "Complete" in selected_graphs else False

#---------------------------------------------------------------------------------------------------------------------------

# Generate the edges for the selected graphs in the edges_folder
generate_graphs_edges(selected_graphs, selected_points, python_executable, edges_folder, points_folder)

# If the complete graph is not selected, we still generate the edges for the complete graph for display and stats purposes
if complete_in_selected_graphs_bool == False:
    generate_graphs_edges(["Complete"], selected_points, python_executable, edges_folder, points_folder)

#---------------------------------------------------------------------------------------------------------------------------
# Normally, the generators are not supposed to generate duplicate edges (a->b and b->a) for the selected graphs

# # Eliminate duplicate edges (a->b and b->a) for the selected graphs
# for graph in selected_graphs:
#     edges_file = f"{edges_folder}/{selected_points.replace('.txt', f'_{graph.lower()}.txt')}"
#     eliminate_duplicate_edges(edges_file)
    
# # Eliminate duplicate edges (a->b and b->a) for the complete graph if it is not selected
# if complete_in_selected_graphs_bool == False:
#     edges_file = f"{edges_folder}/{selected_points.replace('.txt', '_complete.txt')}"
#     eliminate_duplicate_edges(edges_file)
    
# # Save the memory by deleting the variables
# del edges_file

#---------------------------------------------------------------------------------------------------------------------------
# Retrieve the nodes and edges of the complete graph and create the complete graph

# Format txt = name;(x;y)
# Format dic = {name: (x, y)}
nodes_complete = retrieve_nodes(f"{selected_points}", points_folder)

# Format txt = number;name1;name2;weight
# Format dic = {(name1, name2): weight}
edges_complete = retrieve_edges(f"{selected_points.replace('.txt', '_complete.txt')}", edges_folder)

# Construct the complete graph
G_complete = construct_graph(nodes_complete, edges_complete)

#---------------------------------------------------------------------------------------------------------------------------

# Ask the user to select the things to save during the algorithms
selected_things_to_save = select_things_to_save()

# Save the initial graphs if required
if "Initial graphs (GRAPH)" in selected_things_to_save:
    save_initial_graphs_bool = True

In [47]:
# Kruskal algo

save_kruskal_html_bool     = True # Always save the HTML file as it is needed for the stats
save_kruskal_MST_html_bool = True # Always save the HTML file as it is needed for the stats

# Save the kruskal algo as GIF if required
if "Kruskal process (GIF)" in selected_things_to_save:
    save_kruskal_gif_bool = True
    
# Save the MST outputed by kruskal algo if required
if "Kruskal MST (GRAPH)" in selected_things_to_save:
    save_kruskal_MST_bool = True
    
#---------------------------------------------------------------------------------------------------------------------------

to_remove = [] # List of graphs to remove from the selected graphs due to not fully connected graphs

# For every selected type of graph...
# If Complete is not in the selected graphs, we compute it for the HTML but not everything else
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    
    print(f"Processing graph: {graph}")

    # A dictionary to store the edges of the graph
    # Format dic = {(name1, name2): weight}
    graph_edges = retrieve_edges(selected_points.replace('.txt', f'_{graph.lower()}.txt'), edges_folder)
    
    # Construct the graph with the nodes and edges
    # The nodes are the same as the complete graph
    # The edges are the ones of the selected graph
    G_kruskal = construct_graph(nodes=nodes_complete, edges=graph_edges)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Plot the initial graphs if required (save_initial_graphs_bool == True)
    # Prevent to do it for the Complete graph if it is not selected, do it if it is selected
    if save_initial_graphs_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
        
        # Example : AC_30_complete : 435 edges for the AC database with 30 points
        title = selected_points.replace('.txt', f'_{graph.lower()}') + ' : ' + str(len(graph_edges)) + ' edges'
        
        # The base nodes and edges are the ones we display in the background of the graph
        # The base nodes are the ones of the complete graph (same as the selected graph)
        # The base edges are the ones of the selected graph (the ones we want to display)
        
        # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
        # The display is the default one (node size, color, etc.) defined above
        # The plot name is the title defined above
        
        # There are no additional edges or nodes to display in the graph
        # We don't want to make a GIF here so it's set to None
        
        plot_graph(base_nodes=nodes_complete,base_edges=graph_edges,
                   complete_nodes=nodes_complete,complete_edges=edges_complete,
                   plot_name=title,display=display,
                   upper_folder=save_initial_graphs_folder,
                   add_edges_to_display=None, colors_add_edges_to_display=None, label_add_edges_to_display=None,
                   add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
                   gif_bool=None)
        
        # Save the memory by deleting the variables
        del title
        
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Rename the nodes to integers (to work with the kruskal algorithm implementation)
    mapping = {name: i for i, name in enumerate(G_kruskal.nodes)}
    reversed_mapping = {i: name for i, name in enumerate(G_kruskal.nodes)}
    G_kruskal = nx.relabel_nodes(G_kruskal, mapping)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # run the kruskal algorithm step by step
    # it produces : ++
    # - always: an HTML file with the evolution of the MST step by step in the save_kruskal_html_folder
    # - always: an HTML file with the MST outputed by the kruskal algo in the save_kruskal_MST_html_folder
    # - if save_kruskal_gif_bool = True: a GIF file with the evolution of the MST step by step (if required) in the save_kruskal_gif_folder
    # - if save_kruskal_MST_bool = True: a graph with the MST outputed by the kruskal algorithm (if required) in the save_kruskal_MST_folder
    
    # The implementation used needs the graphs with the nodes renamed to integers : G_kruskal
    # The reversed mapping is used to display the true names of the nodes in the different outputs
    # The display is the default one (node size, color, etc.) defined above
    
    # The base nodes and edges are the ones we display in the background of the graph
    # The base nodes are the ones of the complete graph (same as the selected graph)
    # The base edges are the ones of the selected graph (the ones we want to display)
    
    # The graph name is the type of graph we are working on (AC_30_complete, AC_30_RNG, etc.)
    # The accepted edges color code is the one defined above for the accepted edges during the kruskal algo
    # The rejected edges color code is the one defined above for the rejected edges during the kruskal algo
    
    # The other boolean variables are used to save the outputs of the kruskal algo if required in the folders defined above
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # If we treat another graph than the complete one or if the complete graph is selected, we run the kruskal algo with required savings
    if (graph != "Complete" or complete_in_selected_graphs_bool == True):
        accepted_kruskal, rejected_kruskal = kruskal_stepwise(G_kruskal, reversed_mapping=reversed_mapping, display=display,
                                                    base_nodes=nodes_complete, base_edges=graph_edges,
                                                    nodes_complete=nodes_complete, edges_complete=edges_complete,
                                                    graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
                                                    accepted_edges_color_code=accepted_edges_color_code_kruskal,
                                                    rejected_edges_color_code=rejected_edges_color_code_kruskal,
                                                    save_kruskal_gif=save_kruskal_gif_bool, save_kruskal_gif_folder=save_kruskal_gif_folder,
                                                    save_kruskal_html=save_kruskal_html_bool, save_kruskal_html_folder=save_kruskal_html_folder,
                                                    save_kruskal_MST=save_kruskal_MST_bool, save_kruskal_MST_folder=save_kruskal_MST_folder)
        
    # Otherwise, run it but don't save anything except the HTML file
    else:
        accepted_kruskal, rejected_kruskal = kruskal_stepwise(G_kruskal, reversed_mapping=reversed_mapping, display=display,
                                                    base_nodes=nodes_complete, base_edges=graph_edges,
                                                    nodes_complete=nodes_complete, edges_complete=edges_complete,
                                                    graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
                                                    accepted_edges_color_code=accepted_edges_color_code_kruskal,
                                                    rejected_edges_color_code=rejected_edges_color_code_kruskal,
                                                    save_kruskal_gif=False, save_kruskal_gif_folder=None,
                                                    save_kruskal_html=True, save_kruskal_html_folder=save_kruskal_html_folder,
                                                    save_kruskal_MST=False, save_kruskal_MST_folder=None)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Write the MST outputed by the kruskal algo as HTML page
    # We do it even for the complete graph if it is not selected for the stats
    if save_kruskal_MST_html_bool == True:
    
        # Create the folder if it doesn't exist
        if not os.path.exists(save_kruskal_MST_html_folder):
            os.makedirs(save_kruskal_MST_html_folder)
            
        with open(f"{save_kruskal_MST_html_folder}/{selected_points.replace('.txt', f'_{graph.lower()}')}_MST.html", "w") as f:
            
            # The css style and the beginning of the body + the colors
            html_content = f'''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Kruskal Visualization</title>
                <style>
                    .A {{ color: {accepted_edges_color_code_kruskal}; }}
                </style>
            </head>
            '''
            
            # Add the title
            html_content +=  f'''
            <body>
                <h1>Kruskal Visualization for {selected_points.replace('.txt', f'_{graph.lower()}')}</h1>
                <ul>
            '''
            
            #---------------------------------------------------------------------------------------------------------------------------
            
            # Add the edges of the MST outputed by the kruskal algo
            count = 1
            for edge in accepted_kruskal:
                if edge in edges_complete:
                    html_content += f'<li class="A">{count} - Accepted edge: {edge[0]} - {edge[1]} : {edges_complete[edge]}</li>\n'
                    count += 1
                else:
                    html_content += f'<li class="A">{count} - Accepted edge: {edge[0]} - {edge[1]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
                    count += 1
                    
            #---------------------------------------------------------------------------------------------------------------------------
                    
            # Conclude the HTML file        
            html_content += '''
                </ul>
            </body>
            </html>
            '''
            
            # Write the HTML content to the file
            f.write(html_content)
            
            # Save the memory by deleting the variables
            del html_content, count
        
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Save the memory by deleting the variables
    del graph_edges, mapping, reversed_mapping
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # If the MST doesn't countain n-1 edges, eliminate the graph from the list of selected graphs
    # In some cases (as d_radius, KNN), the graph is not connected and the MST doesn't contain n-1 edges
    if len(accepted_kruskal) != len(G_kruskal.nodes) - 1:
        print(f"The graph {graph} has been eliminated from the list of selected graphs.")
        print(f"The MST of the graph {graph} contains {len(accepted_kruskal)} edges instead of {len(G_kruskal.nodes) - 1}.")
        to_remove.append(graph)
        
    # Save the memory by deleting the variables
    del G_kruskal, accepted_kruskal, rejected_kruskal

#---------------------------------------------------------------------------------------------------------------------------

# Remove the graphs that have been eliminated from the list of selected graphs for the rest of the process
for graph in to_remove:
    selected_graphs.remove(graph)
    
#---------------------------------------------------------------------------------------------------------------------------
# For the stats
    
# If a line mentionning the same points file already exists, we delete it
if os.path.exists("variables_needed_for_stats.txt"):
    with open("variables_needed_for_stats.txt", "r") as f2:
        lines = f2.readlines()
    with open("variables_needed_for_stats.txt", "w") as f2:
        for line in lines:
            if str(selected_points).replace('.txt','') not in line:
                f2.write(line)

new_entries = [
    f"selected_points = {selected_points}\n",
    f"selected_graphs_{str(selected_points).replace('.txt', '')} = {selected_graphs}\n",
    f"save_kruskal_html_folder_{str(selected_points).replace('.txt', '')} = {save_kruskal_html_folder}\n",
    f"save_blossom_html_folder_{str(selected_points).replace('.txt', '')} = {save_blossom_html_folder}\n",
    f"save_blossom_matching_html_folder_{str(selected_points).replace('.txt', '')} = {save_blossom_matching_html_folder}\n",
    f"save_shortcutting_tour_html_folder_{str(selected_points).replace('.txt', '')} = {save_shortcutting_tour_html_folder}\n",
    f"save_2opt_final_graph_html_folder_{str(selected_points).replace('.txt', '')} = {save_2opt_final_graph_html_folder}\n"
]

# Read existing entries
try:
    with open("variables_needed_for_stats.txt", "r") as f:
        existing_lines = f.readlines()
except FileNotFoundError:
    existing_lines = []

# Append only non-duplicate entries
with open("variables_needed_for_stats.txt", "a") as f:
    for entry in new_entries:
        if entry not in existing_lines:
            f.write(entry)
   
#---------------------------------------------------------------------------------------------------------------------------

# Save the memory by deleting the variables
del upper_folder_kruskal, upper_folder_hierholzer
del save_initial_graphs_folder, save_initial_graphs_bool
del accepted_edges_color_code_kruskal, rejected_edges_color_code_kruskal
del save_kruskal_gif_bool, save_kruskal_gif_folder
del save_kruskal_html_bool
del save_kruskal_MST_bool, save_kruskal_MST_folder
del save_kruskal_MST_html_bool, save_kruskal_MST_html_folder
del to_remove

Processing graph: RNG
Processing graph: Urquhart
Processing graph: KNN
Processing graph: Delaunay
Processing graph: d_radius
Processing graph: Gabriel
Processing graph: Complete


In [48]:
# Blossom algo

save_blossom_html_bool = True # Always save the HTML file as it is needed for the matching algo and stats
save_blossom_matching_html_bool = True # Always save the HTML file as it is needed for the matching algo and stats

if "Odd degree nodes graphs (GRAPH)" in selected_things_to_save:
    save_blossom_odd_degree_graphs_bool = True
if "Blossom MWPM (GRAPH)" in selected_things_to_save:
    save_blossom_matching_bool = True
if "Superposition MST and MWPM (GRAPH)" in selected_things_to_save:
    save_superposition_bool = True
if "GIF_blossom_matching_process" in selected_things_to_save:
    save_blossom_gif_bool = True
    
#---------------------------------------------------------------------------------------------------------------------------

# For every selected type of graph...    
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    
    print(f"Processing graph: {graph}")
    
    # retrieve the accepted and rejected edges during the kruskal algorithm
    accepted_edges, rejected_edges = retrieve_edges_from_html(f"{save_kruskal_html_folder}/{selected_points.split('.')[0] + '_' + graph.lower()}.html")
    
    #---------------------------------------------------------------------------------------------------------------------------

    # retrieve the degree of each node
    degrees = establish_parity_degree(accepted_edges)
    # keep only the nodes with odd degree
    odd_degrees = {k: v for k, v in degrees.items() if v % 2 != 0}
    # format to keep the same keys as before
    odd_degrees = {k: v for k, v in nodes_complete.items() if k in odd_degrees}

    #---------------------------------------------------------------------------------------------------------------------------

    # Writes the points with odd degree in a txt file in the POINTS folder (the points should be the same for all graphs)
    with open(f'{points_folder}/{selected_points.split(".")[0]}_odd.txt', 'w') as f:
        for name, location in odd_degrees.items():
            f.write(str(name) + ';' + '(' + str(location[0]) + ';' + str(location[1]) + ')' + '\n')

    # Generate the edges of the selected graphs ON THE ODD DEGREE NODES in the EDGES folder
    generate_graphs_edges((selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs),
                          selected_points.split(".")[0] + '_odd.txt', python_executable, edges_folder, points_folder)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Eliminate duplicate edges
    edges_file = f'{edges_folder}/{selected_points.split(".")[0] + "_odd_" + graph.lower()}.txt'
    eliminate_duplicate_edges(edges_file)

    #---------------------------------------------------------------------------------------------------------------------------
        
    # Retrieve the edges for the graph created from the odd degree nodes
    odd_edges = retrieve_edges(selected_points.split(".")[0] + '_odd_' + graph.lower() + '.txt', edges_folder)
    
    # construct the graph with only the odd degree nodes and edges
    ODD = construct_graph(odd_degrees, odd_edges)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # PLot the odd degree nodes graphs if required
    # Prevent to do it for the Complete graph if it is not selected, do it if it is selected
    if save_blossom_odd_degree_graphs_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
        
        # We plot only the nodes and edges of the odd degree graph
        # The number of nodes and edges is displayed in the title of the plot
        
        plot_graph(base_nodes=odd_degrees, base_edges=odd_edges, display=display,
                   complete_nodes=nodes_complete, complete_edges=edges_complete,
                   plot_name=selected_points.replace('.txt', f'_{graph.lower()}_odd') + ' : ' + str(len(odd_degrees)) + ' nodes and ' + str(len(odd_edges)) + ' edges',
                   upper_folder=save_blossom_odd_degree_graphs_folder,
                   add_edges_to_display=None, colors_add_edges_to_display=None, label_add_edges_to_display=None,
                   add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
                   gif_bool=None)
        
    #---------------------------------------------------------------------------------------------------------------------------
    # run the blossom algorithm to find the minimum weight matching
    
    # If there are more than 2 nodes with odd degree, we run the blossom algorithm
    if len(ODD.edges) > 1:
        matching = min_weight_matching(ODD, graph_name =selected_points.replace('.txt', f'_{graph.lower()}'), display=display,
                                       nodes_complete = nodes_complete, edges_complete = edges_complete,
                                       S_color_code_blossom = S_color_code_blossom, T_color_code_blossom = T_color_code_blossom,
                                       visited_color_code_blossom = visited_color_code_blossom,
                                       matching_color_code_blossom = matching_color_code_blossom,
                                       blossom_color_code_blossom = blossom_color_code_blossom,
                                       save_blossom_gif_bool=save_blossom_gif_bool, save_blossom_gif_folder=save_blossom_gif_folder,
                                       save_blossom_html_bool=save_blossom_html_bool, save_blossom_html_folder=save_blossom_html_folder)
                                        
        
    # If there are 2 nodes with odd degree, we directly take the edge between them
    elif len(ODD.edges) == 1:
        matching = [(np.array(list(odd_degrees.keys()))[0], np.array(list(odd_degrees.keys()))[1])]
        
    # Otherwise, there's no edge to take
    else:
        matching = []
        
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Computes the total weight of the matching
    total = 0
    if len(matching) > 0:
        for edge in matching: 
            total += ODD.edges[edge]['weight']
        
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Filter the edges of the matching
    accepted_in_kruskal = []
    rejected_in_kruskal = []
    not_seen_in_kruskal = []
    
    for edge in matching:
        if (edge[0], edge[1]) in accepted_edges or (edge[1], edge[0]) in accepted_edges:
            accepted_in_kruskal.append(edge)
        elif (edge[0], edge[1]) in rejected_edges or (edge[1], edge[0]) in rejected_edges:
            rejected_in_kruskal.append(edge)
        else:
            not_seen_in_kruskal.append(edge)
            
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Write the matching outputed by the blossom algo as HTML page
    # We do it even for the complete graph if it is not selected for the stats
    
    if save_blossom_matching_html_bool == True:
        
        # Create the folder if it doesn't exist
        if not os.path.exists(save_blossom_matching_html_folder):
            os.makedirs(save_blossom_matching_html_folder)
            
        with open(f"{save_blossom_matching_html_folder}/{selected_points.replace('.txt', f'_{graph.lower()}')}_matching.html", "w") as f:
            
            # The css style and the beginning of the body + the colors
            html_content = f'''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Blossom Matching Visualization</title>
                <style>
                    .accepted {{ color: {'#af002a'}; }}
                    .rejected {{ color: {'#2b4fe2'}; }}
                    .not_seen {{ color: {'#f0e58c'}; }}
                </style>
            </head>
            '''
            
            # Add the title
            html_content +=  f'''
            <body>
                <h1>Blossom Matching Visualization for {selected_points.replace('.txt', f'_{graph.lower()}')}</h1>
                <ul>
            '''
            
            #---------------------------------------------------------------------------------------------------------------------------
            
            # Add the edges of the matching outputed by the blossom algo
            count = 1
            for edge in matching:
                if edge in accepted_in_kruskal:
                    if (edge[0], edge[1]) in edges_complete:
                        html_content += f'<li class="accepted">{count} - Accepted edge in Kruskal: {edge[0]} - {edge[1]} : {edges_complete[(edge[0], edge[1])]}</li>\n'
                        count += 1
                    elif (edge[1], edge[0]) in edges_complete:
                        html_content += f'<li class="accepted">{count} - Accepted edge in Kruskal: {edge[1]} - {edge[0]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
                        count += 1
                elif edge in rejected_in_kruskal:
                    if (edge[0], edge[1]) in edges_complete:
                        html_content += f'<li class="rejected">{count} - Rejected edge in Kruskal: {edge[0]} - {edge[1]} : {edges_complete[(edge[0], edge[1])]}</li>\n'
                        count += 1
                    elif (edge[1], edge[0]) in edges_complete:
                        html_content += f'<li class="rejected">{count} - Rejected edge in Kruskal: {edge[1]} - {edge[0]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
                        count += 1
                else:
                    if (edge[0], edge[1]) in edges_complete:
                        html_content += f'<li class="not_seen">{count} - Not seen in Kruskal: {edge[0]} - {edge[1]} : {edges_complete[(edge[0], edge[1])]}</li>\n'
                        count += 1
                    elif (edge[1], edge[0]) in edges_complete:
                        html_content += f'<li class="not_seen">{count} - Not seen in Kruskal: {edge[1]} - {edge[0]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
                        count += 1
                
            #---------------------------------------------------------------------------------------------------------------------------
            
            # Conclude the HTML file        
            html_content += '''
                </ul>
            </body>
            </html>
            '''
            
            # Write the HTML content to the file
            f.write(html_content)
            
            # Save the memory by deleting the variables
            del html_content, count
            
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Plot the matching of the odd degree nodes if required
    # Prevent to do it for the Complete graph if it is not selected, do it if it is selected
    if save_blossom_matching_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
        
        # Here we display the matching of the odd degree nodes with :
        # - the edges accepted in the kruskal algorithm in crimson
        # - the edges rejected in the kruskal algorithm in darkblue
        # - the edges not seen in the kruskal algorithm in gold
        
        edges_to_display = [accepted_in_kruskal, rejected_in_kruskal, not_seen_in_kruskal]
        colors_edges_to_display = ['#af002a', '#2b4fe2', '#f0e58c']
        label_edges_to_display = ['Accepted in Kruskal', 'Rejected in Kruskal', 'Not seen in Kruskal']
        
        # The title of the plot is the name of the selected points file with the type of graph and the total weight of the matching
        title = selected_points.replace('.txt', f'_{graph.lower()}_matching') + ' : ' + str(round(total, 5)) + ' (total weight)'
        
        # Mention if the matching is perfect or not
        if len(matching) != len(odd_degrees) / 2:
            title += '\n The matching is not perfect !'
        
        # The base nodes and edges are the odd degree nodes and edges
        plot_graph(base_nodes=odd_degrees, base_edges=odd_edges, display=display,
                   complete_nodes=nodes_complete, complete_edges=edges_complete,
                   plot_name=title,
                   upper_folder=save_blossom_matching_folder,
                   add_edges_to_display=edges_to_display, colors_add_edges_to_display=colors_edges_to_display, label_add_edges_to_display=label_edges_to_display,
                   add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
                   gif_bool=None)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Computes the total weight of the superposition of the MST and the matching
    total = 0
    
    # For the edges of the MST
    for edge in accepted_edges:
        if (edge[0], edge[1]) in G_complete.edges:
            total += G_complete.edges[(edge[0], edge[1])]['weight']
        elif (edge[1], edge[0]) in G_complete.edges:
            total += G_complete.edges[(edge[1], edge[0])]['weight']
    
    # For the edges of the matching that are not in the MST        
    for edge in matching:
        if (edge[0], edge[1]) in ODD.edges and (edge[0], edge[1]) not in accepted_edges and (edge[1], edge[0]) not in accepted_edges:
            total += ODD.edges[(edge[0], edge[1])]['weight']
        elif (edge[1], edge[0]) in ODD.edges and (edge[0], edge[1]) not in accepted_edges and (edge[1], edge[0]) not in accepted_edges:
            total += ODD.edges[(edge[1], edge[0])]['weight']
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Save the matching and MST in a txt file in the edges folder
    count = 1
    with open(f'{edges_folder}/{selected_points.split(".")[0]}_{graph.lower()}_superposition.txt', 'w') as f:
        
        # Writes the edges of the MST that are not in the matching
        for i, edge in enumerate([edge for edge in accepted_edges if edge not in accepted_in_kruskal and (edge[1], edge[0]) not in accepted_in_kruskal]):
            if (edge[0], edge[1]) in G_complete.edges:
                f.write(str(count) + ';' + str(edge[0]) + ';' + str(edge[1]) + ';' + str(G_complete.edges[(edge[0], edge[1])]['weight']) + '\n')     
                count += 1
            elif (edge[1], edge[0]) in G_complete.edges:
                f.write(str(count) + ';' + str(edge[1]) + ';' + str(edge[0]) + ';' + str(G_complete.edges[(edge[1], edge[0])]['weight']) + '\n')
                count += 1
                     
        # Writes the edges of the matching
        for i, edge in enumerate(matching):
            if (edge[0], edge[1]) in ODD.edges:
                f.write(str(count) + ';' + str(edge[0]) + ';' + str(edge[1]) + ';' + str(ODD.edges[(edge[0], edge[1])]['weight']) + '\n')
                count += 1
            elif (edge[1], edge[0]) in ODD.edges:
                f.write(str(count) + ';' + str(edge[1]) + ';' + str(edge[0]) + ';' + str(ODD.edges[(edge[1], edge[0])]['weight']) + '\n')
                count += 1
                
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Plot the superposition of the MST and the matching if required
    if save_superposition_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
        
        if not os.path.exists(save_superposition_folder):
            os.makedirs(save_superposition_folder)
        
        # Here we display the MST and the matching of the odd degree nodes with :
        # - the edges of the MST that ARE NOT in the matching in violet
        # - the edges of the matching that have been selected in the kruskal algorithm in crimson
        # - the edges of the matching that have been rejected in the kruskal algorithm in darkblue
        # - the edges of the matching that have not been seen in the kruskal algorithm in gold
        
        edges_to_display = [[edge for edge in accepted_edges if edge not in accepted_in_kruskal and (edge[1], edge[0]) not in accepted_in_kruskal],
                            accepted_in_kruskal,rejected_in_kruskal,not_seen_in_kruskal]
        
        colors_edges_to_display = ['#b694fa', '#af002a', '#2b4fe2', '#f0e58c']
        label_edges_to_display = ['MST', 'Matching accepted in Kruskal', 'Matching rejected in Kruskal', 'Matching not seen in Kruskal']
        
        # The title of the plot is the name of the selected points file with the type of graph and the total weight of the superposition
        title = selected_points.replace('.txt', f'_{graph.lower()}_superposition') + ' : ' + str(round(total, 5)) + ' (total weight)'
        
        # Mention if the matching is perfect or not
        if len(matching) != len(odd_degrees) / 2:
            title += '\n The matching is not perfect !'
            
        # A dictionary to store the edges of the graph
        # Format dic = {(name1, name2): weight}
        graph_edges = retrieve_edges(selected_points.replace('.txt', f'_{graph.lower()}_superposition.txt'), edges_folder)
        
        # The base nodes and edges are the nodes of the complete graph and the edges of the selected graph
        plot_graph(base_nodes=nodes_complete, base_edges=graph_edges, display=display,
                   complete_nodes=nodes_complete, complete_edges=edges_complete,
                   plot_name=title,
                   upper_folder=save_superposition_folder,
                   add_edges_to_display=edges_to_display, colors_add_edges_to_display=colors_edges_to_display, label_add_edges_to_display=label_edges_to_display,
                   add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
                   gif_bool=None)
        
        # Save the memory by deleting the variables
        del graph_edges
        
    # Save the memory by deleting the _odd points file
    if os.path.exists(f'{points_folder}/{selected_points.split(".")[0]}_odd.txt'):
        os.remove(f'{points_folder}/{selected_points.split(".")[0]}_odd.txt')
        
# Save the memory by deleting the variables
del G_complete

# Save the memory by deleting the _odd edges file
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    if os.path.exists(f'{edges_folder}/{selected_points.split(".")[0] + "_odd_" + graph.lower()}.txt'):
        os.remove(f'{edges_folder}/{selected_points.split(".")[0] + "_odd_" + graph.lower()}.txt')

Processing graph: RNG
Processing graph: Urquhart
Processing graph: KNN
Processing graph: Delaunay
Processing graph: d_radius
Processing graph: Gabriel
Processing graph: Complete


In [49]:
# Hierholzer algo

save_hierholzer_html_bool                  = True # Always save the HTML file as it is needed for the matching algo and stats
save_hierholzer_eulerian_circuit_html_bool = True # Always save the HTML file as it is needed for the matching algo and stats

# Save the progress of the hierholzer algo as GIF if required
if "Hierholzer process (GIF)" in selected_things_to_save:
    save_hierholzer_gif_bool = True
    
# Save the tour outputed by the hierholzer algo if required
if "Hierholzer tour (GRAPH)" in selected_things_to_save:
    save_hierholzer_eulerian_circuit_bool = True

#---------------------------------------------------------------------------------------------------------------------------

# For every selected type of graph...
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    
    print(f"Processing graph: {graph}")
    
    # retrieve the accepted and rejected edges during the kruskal algorithm
    accepted_edges, rejected_edges = retrieve_edges_from_html(f"{save_kruskal_html_folder}/{selected_points.split('.')[0] + '_' + graph.lower()}.html")

    # Find the last node added to the MST (it's the one for which the first occurrence is in the last edge of the MST)
    last_node_added = accepted_edges[-1][1] if (accepted_edges[-1][0] in {edge[0] for edge in accepted_edges[:-1]}) or (accepted_edges[-1][0] in {edge[1] for edge in accepted_edges[:-1]}) else accepted_edges[-1][0]
    
    # Save the memory by deleting the variables
    del accepted_edges, rejected_edges
     
    #---------------------------------------------------------------------------------------------------------------------------
    
    # retrieve the nodes and edges of the superposition of the MST and the matching for the selected graph
    edges_sup = retrieve_edges(selected_points.replace('.txt', f'_{graph.lower()}_superposition.txt'), f'{edges_folder}/')
    
    # construct the graph with the nodes and edges of the superposition of the MST and the matching
    G_sup = construct_graph(nodes_complete, edges_sup)
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # The implementation used needs the graphs with the nodes : G_sup
    # The source node of the algo is the last node added to the MST (the one for which the first occurrence is in the last edge of the MST)
    # The display is the default one (node size, color, etc.) defined above
    
    # The graph name is the type of graph we are working on (AC_30_complete, AC_30_RNG, etc.)
    # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
    
    # The accepted edges color code is the one defined above for the accepted edges during the hierholzer algo
    # The rejected edges color code is the one defined above for the rejected edges during the hierholzer algo
    
    # The other boolean variables are used to save the outputs of the hierholzer algo if required in the folders defined above
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # If we treat another graph than the complete one or if the complete graph is selected, we run the hierholzer algo
    if graph != "Complete" or complete_in_selected_graphs_bool == True:
        circuit = eulerian_circuit(G_sup, source_node=last_node_added, display=display,
                                graph_edges=edges_sup,
                                graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
                                nodes_complete=nodes_complete, edges_complete=edges_complete,
                                accepted_edges_color_code=accepted_edges_color_code_hierholzer,
                                rejected_edges_color_code=rejected_edges_color_code_hierholzer,
                                save_eulerian_circuit_gif=save_hierholzer_gif_bool, save_eulerian_circuit_gif_folder=save_hierholzer_gif_folder,
                                save_eulerian_circuit_html=save_hierholzer_html_bool, save_eulerian_circuit_html_folder=save_hierholzer_html_folder)
        
    # Otherwise, run it but don't save anything except the HTML file
    else:
        circuit = eulerian_circuit(G_sup, source_node=last_node_added, display=display,
                                graph_edges=edges_sup,
                                graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
                                nodes_complete=nodes_complete, edges_complete=edges_complete,
                                accepted_edges_color_code=accepted_edges_color_code_hierholzer,
                                rejected_edges_color_code=rejected_edges_color_code_hierholzer,
                                save_eulerian_circuit_gif=False, save_eulerian_circuit_gif_folder=None,
                                save_eulerian_circuit_html=True, save_eulerian_circuit_html_folder=save_hierholzer_html_folder)
    
    # Save the memory by deleting the variables
    del last_node_added, G_sup
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Write the eulerian circuit as HTML page
    # We do it even for the complete graph if it is not selected for the stats
    if save_hierholzer_eulerian_circuit_html_bool == True:
        
        # Create the folder if it doesn't exist
        if not os.path.exists(save_hierholzer_eulerian_circuit_html_folder):
            os.makedirs(save_hierholzer_eulerian_circuit_html_folder)
            
        with open(f"{save_hierholzer_eulerian_circuit_html_folder}/{selected_points.replace('.txt', f'_{graph.lower()}')}_eulerian_circuit.html", "w") as f:
            
            # The css style and the beginning of the body + the colors
            html_content = f'''
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>Hierholzer Visualization</title>
                <style>
                    .A {{ color: {accepted_edges_color_code_hierholzer}; }}
                </style>
            </head>
            '''
            
            # Add the title
            html_content +=  f'''
            <body>
                <h1>Hierholzer Visualization for {selected_points.replace('.txt', f'_{graph.lower()}')}</h1>
                <ul>
            '''
            
            #---------------------------------------------------------------------------------------------------------------------------
            
            # Add the edges of the eulerian circuit
            count = 1
            for edge in circuit:
                if (edge[0], edge[1]) in edges_complete:
                    html_content += f'<li class="A">{count} - Accepted edge: {edge[0]} - {edge[1]} : {edges_complete[edge]}</li>\n'
                    count += 1
                    
                else:
                    html_content += f'<li class="A">{count} - Accepted edge: {edge[0]} - {edge[1]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
                    count += 1
                    
            #---------------------------------------------------------------------------------------------------------------------------
            
            # Conclude the HTML file
            html_content += '''
                </ul>
            </body>
            </html>
            '''
            
            # Write the HTML content to the file
            f.write(html_content)
            
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Write the edges of the eulerian circuit in a txt file in the edges_folder with _circuit at the end of the name
    # The format is the same as the one used for the edges of the superposition of the MST and the matching
    with open(f'{edges_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_circuit.txt")}', 'w') as f:
        
        # For each edge in the circuit...
        for i, edge in enumerate(circuit):
            
            # If the edge is in the edges_complete dictionary, we write it in the file with the format : number;name1;name2;weight
            if (edge[0], edge[1]) in edges_complete:
                f.write(str(i + 1) + ';' + str(edge[0]) + ';' + str(edge[1]) + ';' + str(edges_complete[(edge[0], edge[1])]) + '\n')
                
            # If the edge is in the edges_complete dictionary but in the reverse order, we write it in the file with the format : number;name1;name2;weight
            elif (edge[1], edge[0]) in edges_complete:
                f.write(str(i + 1) + ';' + str(edge[1]) + ';' + str(edge[0]) + ';' + str(edges_complete[(edge[1], edge[0])]) + '\n')
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Compute the total weight of the eulerian circuit
    total = 0
    for e in circuit :
        if (e[0], e[1]) in edges_complete:
            total += edges_complete[(e[0], e[1])]
        elif (e[1], e[0]) in edges_complete:
            total += edges_complete[(e[1], e[0])]
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Verify that the eulerian circuit is possible
    if circuit == None:
        print('No eulerian circuit found for the graph ' + selected_points.replace('.txt', f'_{graph.lower()}_superposition') + ' !')
        
    else:
        
        # Save the eulerian circuit in a txt file in the save_hierholzer_eulerian_circuit_folder if required
        # Prevent to do it for the Complete graph if it is not selected, do it if it is selected
        if save_hierholzer_eulerian_circuit_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
            
            # Example : AC_30_delaunay_eulerian_circuit : 76.80527 (total weight) 
            title = selected_points.replace('.txt', f'_{graph.lower()}_eulerian_circuit') + ' : ' + str(round(total, 5)) + ' (total weight)'
            
            # The base nodes and edges are the ones we display in the background of the graph
            # The base nodes are the ones of the complete graph (same as the selected graph)
            # The base edges are the ones of the superposition of the MST and the matching based on the selected graph
            
            # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
            # The display is the default one (node size, color, etc.) defined above
            # The plot name is the title defined above
            
            # The additional edges to display are the edges of the eulerian circuit
            # The color code is the one defined above for the accepted edges during the hierholzer algo
            # The label is 'eulerian circuit'
            
            # There are no additional nodes to display in the graph
            # We don't want to make a GIF here so it's set to None  
            
            plot_graph(base_nodes=nodes_complete, base_edges=edges_sup, display=display,
                       complete_nodes=nodes_complete, complete_edges=edges_complete,
                       plot_name=title,
                       upper_folder=save_hierholzer_eulerian_circuit_folder,
                       add_edges_to_display=[circuit], colors_add_edges_to_display=[accepted_edges_color_code_hierholzer], label_add_edges_to_display=['Eulerian circuit'],
                       add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
                       gif_bool=None)
            
            # Save the memory by deleting the variables
            del title
            
    #---------------------------------------------------------------------------------------------------------------------------
            
    # Save the memory by deleting the variables
    del edges_sup, circuit, total
    
#---------------------------------------------------------------------------------------------------------------------------
    
# Save the memory by deleting the variables
del accepted_edges_color_code_hierholzer, rejected_edges_color_code_hierholzer
del save_hierholzer_gif_bool, save_hierholzer_gif_folder
del save_hierholzer_html_bool
del save_hierholzer_eulerian_circuit_bool, save_hierholzer_eulerian_circuit_folder
del save_kruskal_html_folder
del save_hierholzer_eulerian_circuit_html_bool, save_hierholzer_eulerian_circuit_html_folder

#---------------------------------------------------------------------------------------------------------------------------

# Save the memory by deleting the _superposition edges file
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    if os.path.exists(f'{edges_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_superposition.txt")}'):
        os.remove(f'{edges_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_superposition.txt")}')

Processing graph: RNG
Processing graph: Urquhart
Processing graph: KNN
Processing graph: Delaunay
Processing graph: d_radius
Processing graph: Gabriel
Processing graph: Complete


In [50]:
# Shortcutting algo

save_shortcutting_html_bool      = True # Always save the HTML file as it is needed for the matching algo and stats
save_shortcutting_tour_html_bool = True # Always save the HTML file as it is needed for the matching algo and stats

if "Shortcutting process (GIF)" in selected_things_to_save:
    save_shortcutting_gif_bool = True
if "Shortcutting tour (GRAPH)" in selected_things_to_save:
    save_shortcutting_tour_bool = True
    
#---------------------------------------------------------------------------------------------------------------------------

# For every selected type of graph...    
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    
    print(f"Processing graph: {graph}")
    
    # Retrieve the edges from the circuit outputed by the hierholzer algorithm
    edges_circuit = retrieve_edges(f'{selected_points.replace(".txt", f"_{graph.lower()}_circuit.txt")}', edges_folder)
    
    # Retrieve the accepted and rejected edges during the hierholzer algorithm
    accepted_edges, _ = retrieve_edges_from_html(f"{save_hierholzer_html_folder}/{selected_points.split('.')[0] + '_' + graph.lower()}.html")
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # The implementation used needs the edges of the circuit outputed by the hierholzer algorithm : accepted_edges
    # The display is the default one (node size, color, etc.) defined above
    # The graph name is the type of graph we are working on (AC_30_complete, AC_30_RNG, etc.) 
    
    # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
    # The base nodes and edges are the ones we display in the background of the graph
    # The base nodes are the ones of the complete graph (same as the selected graph)
    # The base edges are the ones of the circuit outputed by the hierholzer algorithm based on the selected graph
    
    # The accepted edges color code is the one defined above for the accepted edges during the shortcutting algo
    # The rejected edges color code is the one defined above for the rejected edges during the shortcutting algo
    
    # The other boolean variables are used to save the outputs of the shortcutting algo if required in the folders defined above
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # If we treat another graph than the complete one or if the complete graph is selected, we run the shortcutting algo
    if graph != "Complete" or complete_in_selected_graphs_bool == True:
        shortcutted_circuit = shortcutting(accepted_edges, display=display,
                                        base_nodes=nodes_complete, base_edges=edges_circuit,
                                        nodes_complete=nodes_complete, edges_complete=edges_complete,
                                        graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
                                        accepted_edges_color_code=accepted_edges_color_code_shortcutting,
                                        rejected_edges_color_code=rejected_edges_color_code_shortcutting,
                                        save_shortcutting_gif=save_shortcutting_gif_bool, save_shortcutting_gif_folder=save_shortcutting_gif_folder,
                                        save_shortcutting_html=save_shortcutting_html_bool, save_shortcutting_html_folder=save_shortcutting_html_folder)
        
    # Otherwise, run it but don't save anything except the HTML file
    else:
        shortcutted_circuit = shortcutting(accepted_edges, display=display,
                                        base_nodes=nodes_complete, base_edges=edges_circuit,
                                        nodes_complete=nodes_complete, edges_complete=edges_complete,
                                        graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
                                        accepted_edges_color_code=accepted_edges_color_code_shortcutting,
                                        rejected_edges_color_code=rejected_edges_color_code_shortcutting,
                                        save_shortcutting_gif=False, save_shortcutting_gif_folder=None,
                                        save_shortcutting_html=True, save_shortcutting_html_folder=save_shortcutting_html_folder)
    
    # Save the memory by deleting the variables
    del accepted_edges
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Write the edges of the shortcutted circuit in a txt file in the points_folder with _shortcutted_circuit at the end of the name
    # The format is the same as the one used for the edges of the superposition of the MST and the matching
    with open(f'{points_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_shortcutted_circuit.txt")}', 'w') as f:
        for node in shortcutted_circuit:
            f.write(str(node) + ';(' + str(nodes_complete[node][0]) + ';' + str(nodes_complete[node][1]) + ')' + '\n')
            
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Format the shortcutted circuit
    shortcutted_circuit = [(shortcutted_circuit[i], shortcutted_circuit[i + 1]) for i in range(len(shortcutted_circuit) - 1)]
    
    # Compute the total weight of the shortcutted circuit
    total = 0
    for edge in shortcutted_circuit:
        if edge in edges_complete:
            total += edges_complete[edge]
        elif (edge[1], edge[0]) in edges_complete:
            total += edges_complete[(edge[1], edge[0])]
            
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Write the shortcutted circuit as HTML page
    # We do it even for the complete graph if it is not selected for the stats
    if save_shortcutting_html_bool == True:
            
            # Create the folder if it doesn't exist
            if not os.path.exists(save_shortcutting_tour_html_folder):
                os.makedirs(save_shortcutting_tour_html_folder)
                
            with open(f"{save_shortcutting_tour_html_folder}/{selected_points.replace('.txt', f'_{graph.lower()}')}_shortcutted_circuit.html", "w") as f:
                
                # The css style and the beginning of the body + the colors
                html_content = f'''
                <!DOCTYPE html>
                <html lang="en">
                <head>
                    <meta charset="UTF-8">
                    <meta name="viewport" content="width=device-width, initial-scale=1.0">
                    <title>Shortcutting Visualization</title>
                    <style>
                        .A {{ color: {accepted_edges_color_code_shortcutting}; }}
                    </style>
                </head>
                '''
                
                # Add the title
                html_content +=  f'''
                <body>
                    <h1>Shortcutting Visualization for {selected_points.replace('.txt', f'_{graph.lower()}')}</h1>
                    <ul>
                '''
                
                #---------------------------------------------------------------------------------------------------------------------------
                
                # Add the edges of the shortcutted circuit
                count = 1
                for edge in shortcutted_circuit:
                    if (edge[0], edge[1]) in edges_complete:
                        html_content += f'<li class="A">{count} - Accepted edge: {edge[0]} - {edge[1]} : {edges_complete[edge]}</li>\n'
                        count += 1
                    elif (edge[1], edge[0]) in edges_complete:
                        html_content += f'<li class="A">{count} - Accepted edge: {edge[1]} - {edge[0]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
                        count += 1
                        
                #---------------------------------------------------------------------------------------------------------------------------
                
                # Conclude the HTML file
                html_content += '''
                    </ul>
                </body>
                </html>
                '''
                
                # Write the HTML content to the file
                f.write(html_content)
            
    #---------------------------------------------------------------------------------------------------------------------------
    
    # Save the shortcutted circuit if required
    # Prevent to do it for the Complete graph if it is not selected, do it if it is selected
    if save_shortcutting_tour_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
        
        # Example : AC_30_delaunay_eulerian_circuit_shortcut : 75.94559 (total weight)
        title = selected_points.replace('.txt', f'_{graph.lower()}_eulerian_circuit_shortcut') + ' : ' + str(round(total, 5)) + ' (total weight)'
        
        # The base nodes and edges are the ones we display in the background of the graph
        # The base nodes are the ones of the complete graph (same as the selected graph)
        # The base edges are the ones of the circuit outputed by the hierholzer algorithm based on the selected graph
        # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
        
        # The display is the default one (node size, color, etc.) defined above
        # The plot name is the title defined above
        
        # The additional edges to display are the edges of the shortcutted circuit
        # The color code is the one defined above for the accepted edges during the shortcutting algo
        # The label is 'eulerian circuit'
        
        # There are no additional nodes to display in the graph
        # We don't want to make a GIF here so it's set to None
        
        plot_graph(base_nodes=nodes_complete, base_edges=edges_circuit, display=display,
                   complete_nodes=nodes_complete, complete_edges=edges_complete,
                   plot_name=title,
                   upper_folder=save_shortcutting_tour_folder,
                   add_edges_to_display=[shortcutted_circuit], colors_add_edges_to_display=[accepted_edges_color_code_shortcutting], label_add_edges_to_display=['Eulerian circuit'],
                   add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
                   gif_bool=None)
        
        # Save the memory by deleting the variables
        del title
        
    #---------------------------------------------------------------------------------------------------------------------------
        
    # Save the memory by deleting the variables
    del edges_circuit, shortcutted_circuit, total
    
# Save the memory by deleting the _circuit edges file
for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    if os.path.exists(f'{edges_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_circuit.txt")}'):
        os.remove(f'{edges_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_circuit.txt")}')
        
# Save the memory by deleting the variables
del save_hierholzer_html_folder, rejected_edges_color_code_shortcutting
del save_shortcutting_gif_bool, save_shortcutting_gif_folder
del save_shortcutting_html_bool, save_shortcutting_html_folder
del save_shortcutting_tour_bool, save_shortcutting_tour_folder
del edges_folder

Processing graph: RNG
Processing graph: Urquhart
Processing graph: KNN
Processing graph: Delaunay
Processing graph: d_radius
Processing graph: Gabriel
Processing graph: Complete


In [51]:
# # 2-opt algo

# save_2opt_html_bool             = True # Always save the HTML file as it is needed for the matching algo and stats
# save_2opt_final_graph_html_bool = True # Always save the HTML file as it is needed for the matching algo and stats

# if "2-opt process (GIF)" in selected_things_to_save:
#     save_2opt_gif_bool = True
# if "Final tour (GRAPH)" in selected_things_to_save:
#     save_2opt_final_graph_bool = True
    
# # Save the memory by deleting the variables
# del selected_things_to_save
    
# #---------------------------------------------------------------------------------------------------------------------------

# # For every selected type of graph...
# for graph in (selected_graphs + ["Complete"] if complete_in_selected_graphs_bool == False else selected_graphs):
    
#     # Retrieve the nodes from the shortcutted circuit (their position that are required for the 2-opt algo)
#     nodes_shortcutted_circuit = retrieve_nodes(f'{selected_points.replace(".txt", f"_{graph.lower()}_shortcutted_circuit.txt")}', points_folder)
#     nodes_shortcutted_circuit = np.array(list(nodes_shortcutted_circuit.keys()))
#     node_list = [nodes_complete[node] for node in nodes_shortcutted_circuit]
    
#     #---------------------------------------------------------------------------------------------------------------------------
    
#     # The implementation used needs the nodes of the shortcutted circuit : node_list
#     # The display is the default one (node size, color, etc.) defined above
    
#     # The graph name is the type of graph we are working on (AC_30_complete, AC_30_RNG, etc.)
#     # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
    
#     # The base nodes and edges are the ones we display in the background of the graph
#     # The base nodes are the ones of the complete graph (same as the selected graph)
#     # The base edges are None (because displaying the edges of the shortcutted circuit is enough)
    
#     # The shortcutting edges color code is the one defined above for the edges of the shortcutted circuit
#     # The crossing edges color code is the one defined above for the edges of the crossing edges during the 2-opt algo
#     # The corrected edges color code is the one defined above for the edges of the 'uncrossed' edges during the 2-opt algo
    
#     # The other boolean variables are used to save the outputs of the 2-opt algo if required in the folders defined above
    
#     #---------------------------------------------------------------------------------------------------------------------------
    
#     # If we treat another graph than the complete one or if the complete graph is selected, we run the 2-opt algo
#     if graph != "Complete" or complete_in_selected_graphs_bool == True:
#         final_circuit = optimize_tour(node_list, display=display,
#                                     base_nodes=nodes_complete, base_edges={},
#                                     graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
#                                     nodes_complete=nodes_complete, edges_complete=edges_complete,
#                                     shortcutting_edges_color_code=accepted_edges_color_code_shortcutting,
#                                     crossing_edges_color_code=crossing_edges_color_code_2opt,
#                                     corrected_edges_color_code=corrected_edges_color_code_2opt,
#                                     save_two_opt_html=save_2opt_html_bool, save_two_opt_html_folder=save_2opt_html_folder,
#                                     save_two_opt_gif=save_2opt_gif_bool, save_two_opt_gif_folder=save_2opt_gif_folder)
        
#     # Otherwise, run it but don't save anything except the HTML file
#     else:
#         final_circuit = optimize_tour(node_list, display=display,
#                                     base_nodes=nodes_complete, base_edges={},
#                                     graph_name=selected_points.replace('.txt', f'_{graph.lower()}'),
#                                     nodes_complete=nodes_complete, edges_complete=edges_complete,
#                                     shortcutting_edges_color_code=accepted_edges_color_code_shortcutting,
#                                     crossing_edges_color_code=crossing_edges_color_code_2opt,
#                                     corrected_edges_color_code=corrected_edges_color_code_2opt,
#                                     save_two_opt_html=True, save_two_opt_html_folder=save_2opt_html_folder,
#                                     save_two_opt_gif=False, save_two_opt_gif_folder=None)
    
#     # Save the memory by deleting the variables
#     del node_list
    
#     #---------------------------------------------------------------------------------------------------------------------------
    
#     # Construct the edges based on the succesion of points in the final circuit
#     final_tour = []
#     for i in range(len(final_circuit) - 1):
        
#         # Find the key corresponding to the value (final_circuit is a succesion of tuples (x, y))
#         key1 = list(nodes_complete.keys())[list(nodes_complete.values()).index(final_circuit[i])]
#         key2 = list(nodes_complete.keys())[list(nodes_complete.values()).index(final_circuit[i + 1])]
        
#         # Add the edge to the list of edges to display
#         final_tour.append((key1, key2))
        
#     # Don't forget the last edge to close the circuit
#     final_tour.append((key2, final_tour[0][0]))
    
#     # Save the memory by deleting the variables
#     del final_circuit, key1, key2
    
#     #---------------------------------------------------------------------------------------------------------------------------
    
#     # Compute the total weight of the final circuit
#     total = 0
#     for edge in final_tour:
#         if edge in edges_complete:
#             total += edges_complete[edge]
#         elif (edge[1], edge[0]) in edges_complete:
#             total += edges_complete[(edge[1], edge[0])]
            
#     #---------------------------------------------------------------------------------------------------------------------------
     
#     # If we want to save the final circuit...
#     # Prevent to do it for the Complete graph if it is not selected, do it if it is selected
#     if save_2opt_final_graph_bool == True and (graph != "Complete" or complete_in_selected_graphs_bool == True):
        
#         # Rewrite the edges of the final circuit as a dictionary
#         # Format dic = {(name1, name2): weight}
#         circuit_edges = {}
#         for i in range (len(nodes_shortcutted_circuit) - 1):
            
#             # If the edge is in the edges_complete dictionary, we add it to the circuit_edges dictionary
#             if (nodes_shortcutted_circuit[i], nodes_shortcutted_circuit[i + 1]) in edges_complete:
#                 circuit_edges[(nodes_shortcutted_circuit[i], nodes_shortcutted_circuit[i + 1])] = edges_complete[(nodes_shortcutted_circuit[i], nodes_shortcutted_circuit[i + 1])]
            
#             # If the edge is in the edges_complete dictionary but in the reverse order, we add it to the circuit_edges dictionary
#             elif (nodes_shortcutted_circuit[i + 1], nodes_shortcutted_circuit[i]) in edges_complete:
#                 circuit_edges[(nodes_shortcutted_circuit[i], nodes_shortcutted_circuit[i + 1])] = edges_complete[(nodes_shortcutted_circuit[i + 1], nodes_shortcutted_circuit[i])]
        
#         #---------------------------------------------------------------------------------------------------------------------------
            
#         # Example : AC_30_delaunay_eulerian_circuit_shortcut_optimized : 71.59661 (total weight)
#         title = selected_points.replace('.txt', f'_{graph.lower()}_eulerian_circuit_shortcut_optimized') + ' : ' + str(round(total, 5)) + ' (total weight)'
        
#         # The base nodes and edges are the ones we display in the background of the graph
#         # The base nodes are the ones of the complete graph (same as the selected graph)
#         # The base edges are the ones of the shortcutted circuit based on the selected graph
        
#         # The complete nodes and edges are used for display purposes and to compute the weights of the edges in the process
#         # The display is the default one (node size, color, etc.) defined above
#         # The plot name is the title defined above
        
#         # The additional edges to display are the edges of the final circuit
#         # The color code is the one defined above for the edges of the final circuit during the 2-opt algo
#         # The label is 'Final circuit'
        
#         # There are no additional nodes to display in the graph
#         # We don't want to make a GIF here so it's set to None
        
#         plot_graph(base_nodes=nodes_complete, base_edges=circuit_edges, display=display,
#                 complete_nodes=nodes_complete, complete_edges=edges_complete,
#                 plot_name=title,
#                 upper_folder=save_2opt_final_graph_folder,
#                 add_edges_to_display=[final_tour], colors_add_edges_to_display=[tour_edges_color_code_2opt], label_add_edges_to_display=['Final circuit'],
#                 add_nodes_to_display=None, colors_add_nodes_to_display=None, label_add_nodes_to_display=None,
#                 gif_bool=None)
        
#         # Save the memory by deleting the variables
#         del circuit_edges, title
        
#     #---------------------------------------------------------------------------------------------------------------------------
    
#     # Write the edges of the final circuit as HTML
#     if save_2opt_final_graph_html_bool == True:
        
#         # Create the folder if it doesn't exist
#         if not os.path.exists(save_2opt_final_graph_html_folder):
#             os.makedirs(save_2opt_final_graph_html_folder)
        
#         with open(f'{save_2opt_final_graph_html_folder}/{selected_points.replace(".txt", f"_{graph.lower()}_final_circuit.html")}', 'w') as f:
            
                
#                 # The css style and the beginning of the body + the colors
#                 html_content = f'''
#                 <!DOCTYPE html>
#                 <html lang="en">
#                 <head>
#                     <meta charset="UTF-8">
#                     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#                     <title>2-opt Visualization</title>
#                     <style>
#                         .A {{ color: {tour_edges_color_code_2opt}; }}
#                     </style>
#                 </head>
#                 '''
                
#                 # Add the title
#                 html_content +=  f'''
#                 <body>
#                     <h1>2-opt Visualization for {selected_points.replace('.txt', f'_{graph.lower()}')}</h1>
#                     <ul>
#                 '''
                
#                 #---------------------------------------------------------------------------------------------------------------------------
                
#                 # Add the edges of the final circuit
#                 count = 1
#                 for edge in final_tour:
#                     if (edge[0], edge[1]) in edges_complete:
#                         html_content += f'<li class="A">{count} - Accepted edge: {edge[0]} - {edge[1]} : {edges_complete[edge]}</li>\n'
#                         count += 1
#                     elif (edge[1], edge[0]) in edges_complete:
#                         html_content += f'<li class="A">{count} - Accepted edge: {edge[1]} - {edge[0]} : {edges_complete[(edge[1], edge[0])]}</li>\n'
#                         count += 1
                        
#                 #---------------------------------------------------------------------------------------------------------------------------
                
#                 # Conclude the HTML file
#                 html_content += '''
#                     </ul>
#                 </body>
#                 </html>
#                 '''
                
#                 # Write the HTML content to the file
#                 f.write(html_content)
                    
#     #---------------------------------------------------------------------------------------------------------------------------
    
#     # Save the memory by deleting the variables
#     del nodes_shortcutted_circuit, final_tour, total
    
#     # Save the memory by deleting the _shortcutted_circuit file
#     if os.path.exists(f'{points_folder}/{selected_points.split(".")[0]}_{graph.lower()}_shortcutted_circuit.txt'):
#         os.remove(f'{points_folder}/{selected_points.split(".")[0]}_{graph.lower()}_shortcutted_circuit.txt')
        
# # Save the memory by deleting the variables
# del selected_points, selected_graphs, nodes_complete, edges_complete, display
# del accepted_edges_color_code_shortcutting
# del tour_edges_color_code_2opt, crossing_edges_color_code_2opt, corrected_edges_color_code_2opt
# del save_2opt_gif_bool, save_2opt_gif_folder
# del save_2opt_html_bool, save_2opt_html_folder
# del save_2opt_final_graph_bool, save_2opt_final_graph_folder
# del points_folder

In [52]:
# delete all __pycache__ folders
folders = ['AC', 'Blossom', 'Graph_generators', 'Graph_visualizers', 'Hierholzer', 'Kruskal', 'Point_maker', 'Shortcutting', 'two_opt', 'utils']

for folder in folders:
    if os.path.exists(folder):
        for file in os.listdir(folder):
            if file == '__pycache__':
                for elem in os.listdir(folder + '/' + file):
                    os.remove(folder + '/' + file + '/' + elem)
                os.rmdir(folder + '/' + file)